# Solutions for the generator exercise

⚠️ **SOLUTIONS:** This notebook contains example solutions for the [generator exercise](./03a_exercise1.ipynb)

## 1. Imports

In [1]:
import simpy
import numpy as np

## Example code

The code below is taken from the simple call centre example.  In this code arrivals occur with an inter-arrival time (IAT) of exactly 1 minute.

In [2]:
def arrivals_generator(env):
    '''
    Prescriptions arrive with a fixed duration of 1 minute.

    Parameters:
    ------
    env: simpy.Environment
    '''
    
    # don't worry about the infinite while loop, simpy will
    # exit at the correct time.
    while True:
        
        # sample an inter-arrival time.
        inter_arrival_time = 1.0
        
        # we use the yield keyword instead of return
        yield env.timeout(inter_arrival_time)
        
        # print out the time of the arrival
        print(f'Call arrives at: {env.now}')

In [3]:
# model parameters
RUN_LENGTH = 25

# create the simpy environment object
env = simpy.Environment()

# tell simpy that the `arrivals_generator` is a process
env.process(arrivals_generator(env))

# run the simulation model
env.run(until=RUN_LENGTH)
print(f'end of run. simulation clock time = {env.now}')

Call arrives at: 1.0
Call arrives at: 2.0
Call arrives at: 3.0
Call arrives at: 4.0
Call arrives at: 5.0
Call arrives at: 6.0
Call arrives at: 7.0
Call arrives at: 8.0
Call arrives at: 9.0
Call arrives at: 10.0
Call arrives at: 11.0
Call arrives at: 12.0
Call arrives at: 13.0
Call arrives at: 14.0
Call arrives at: 15.0
Call arrives at: 16.0
Call arrives at: 17.0
Call arrives at: 18.0
Call arrives at: 19.0
Call arrives at: 20.0
Call arrives at: 21.0
Call arrives at: 22.0
Call arrives at: 23.0
Call arrives at: 24.0
end of run. simulation clock time = 25


## 3. Exercise: Modelling a poisson arrival process for prescriptions

**Task:**

Update `arrivals_generator()` so that inter-arrival times follow an **exponential distribution** with a mean inter-arrival time of 60.0 / 100 minutes between arrivals (i.e. 100 arrivals per hour). Use a run length of 25 minutes.

**Bonus challenge:**

* First, try implementing this **without** setting a random seed.
* Then, update the method with an approach to control the randomness,

**Hints:**

* We learnt how to sample using a `numpy` random number generator in the [sampling notebook](./01_sampling.ipynb). Excluding a random seed, the basic method for drawing a single sample follows this pattern:
    ```python
    rng = np.random.default_rng()
    sample = rng.exponential(scale=12.0)
    ```

### 3.1 Example answer 1

In [4]:
# example answer
def arrivals_generator(env, random_seed=None):
    '''
    Time between caller arrivals follows an Expoential distribution with mean
    inter-arrival time of 60.0/100.0 minutes
    
    Parameters:
    ------
    env: simpy.Environment
    
    random_state: int, optional (default=None)
        if set then used as random seed to control sampling.
    '''
    rs_arrivals = np.random.default_rng(random_seed)
    
    while True:
        inter_arrival_time = rs_arrivals.exponential(60.0/100.0)
        yield env.timeout(inter_arrival_time)
        print(f'Call arrives at: {env.now}')

In [5]:
# model parameters
RUN_LENGTH = 25

# create the simpy environment object
env = simpy.Environment()

# tell simpy that the `arrivals_generator` is a process
env.process(arrivals_generator(env))

# run the simulation model
env.run(until=RUN_LENGTH)
print(f'end of run. simulation clock time = {env.now}')

Call arrives at: 1.2047204928588542
Call arrives at: 1.665546474948937
Call arrives at: 2.373859767681621
Call arrives at: 2.8557180455305122
Call arrives at: 3.292413414302393
Call arrives at: 3.948584404430685
Call arrives at: 4.635827332239991
Call arrives at: 4.713197604859726
Call arrives at: 6.2100100085567185
Call arrives at: 6.774734821423559
Call arrives at: 6.793089740386549
Call arrives at: 7.723100804981457
Call arrives at: 9.071806936287263
Call arrives at: 9.883018879630491
Call arrives at: 10.292246335013177
Call arrives at: 11.206429085014648
Call arrives at: 12.861416213324771
Call arrives at: 13.710937641648242
Call arrives at: 14.510705172538367
Call arrives at: 14.625813856487689
Call arrives at: 15.61718210481707
Call arrives at: 16.296921355929992
Call arrives at: 17.41307760431281
Call arrives at: 19.228422092191522
Call arrives at: 19.440896829258076
Call arrives at: 19.64343927716942
Call arrives at: 20.588121606767952
Call arrives at: 20.985618515608884
Call a

### 3.2 Example answer 2

In this solution, we first define a class called `Exponential` and pass that as an argument to the generator.

In [6]:
class Exponential:
    '''
    Convenience class for the exponential distribution.
    packages up distribution parameters, seed and random generator.
    '''
    def __init__(self, mean, random_seed=None):
        '''
        Constructor

        Params:
        ------
        mean: float
            The mean of the exponential distribution

        random_seed: int, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        '''
        self.rand = np.random.default_rng(seed=random_seed)
        self.mean = mean

    def sample(self, size=None):
        '''
        Generate a sample from the exponential distribution

        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.
        '''
        return self.rand.exponential(self.mean, size=size)

In [7]:
# example answer
def arrivals_generator(env, iat_dist):
    '''
    Call arrival process. Calls follow a user specified distribution
    
    Parameters:
    ------
    env: simpy.Environment
    
    iat_dist: object
        A python class that implements a .sample() method
        and generates the IATs
    '''        
    while True:
        inter_arrival_time = iat_dist.sample()
        yield env.timeout(inter_arrival_time)
        print(f'Prescription arrives at: {env.now}')

In [8]:
# model parameters
RUN_LENGTH = 25

# create the simpy environment object
env = simpy.Environment()

iat = Exponential(mean=60.0 / 100.0, random_seed=42)

# tell simpy that the `arrivals_generator` is a process
env.process(arrivals_generator(env, iat))

# run the simulation model
env.run(until=RUN_LENGTH)
print(f'end of run. simulation clock time = {env.now}')

Prescription arrives at: 1.4425251623795967
Prescription arrives at: 2.8442389558742684
Prescription arrives at: 4.2750955557988215
Prescription arrives at: 4.442972129709771
Prescription arrives at: 4.4948345695287975
Prescription arrives at: 5.366430878952488
Prescription arrives at: 6.212407295507032
Prescription arrives at: 8.08698486943673
Prescription arrives at: 8.134561387746835
Prescription arrives at: 8.762497895672734
Prescription arrives at: 8.804759679751418
Prescription arrives at: 9.458173856565795
Prescription arrives at: 10.49697026829608
Prescription arrives at: 10.729107163050026
Prescription arrives at: 11.46805856064697
Prescription arrives at: 11.560322512942706
Prescription arrives at: 11.615268873376822
Prescription arrives at: 11.804376394656257
Prescription arrives at: 12.345095959878352
Prescription arrives at: 12.592887157721446
Prescription arrives at: 13.34131872760962
Prescription arrives at: 13.475464550937012
Prescription arrives at: 14.57824670942369
P

### 3.3 Why would we use solution 2?

Solution 2 is a useful approach as it is now **easy to define new experiments**.

We could experiment with the mean of the exponential or use an entirely different distribution (as long as it implements `.sample()`) without changing our generator function. 

For example...

In [9]:
# model parameters
RUN_LENGTH = 25

# create the simpy environment object
env = simpy.Environment()

# ****** MODIFICATION: reduce IAT. ******
# Note: with this method we could use a different Exponetial parameters or
# even a different distribution such as Erlang
iat = Exponential(mean=60.0 / 80.0, random_seed=42)

# tell simpy that the `arrivals_generator` is a process
env.process(arrivals_generator(env, iat))

# run the simulation model
env.run(until=RUN_LENGTH)
print(f'end of run. simulation clock time = {env.now}')

Prescription arrives at: 1.8031564529744961
Prescription arrives at: 3.5552986948428362
Prescription arrives at: 5.3438694447485275
Prescription arrives at: 5.553715162137215
Prescription arrives at: 5.618543211910998
Prescription arrives at: 6.708038598690611
Prescription arrives at: 7.7655091193837915
Prescription arrives at: 10.108731086795915
Prescription arrives at: 10.168201734683548
Prescription arrives at: 10.95312236959092
Prescription arrives at: 11.005949599689277
Prescription arrives at: 11.822717320707246
Prescription arrives at: 13.121212835370102
Prescription arrives at: 13.411383953812535
Prescription arrives at: 14.335073200808715
Prescription arrives at: 14.450403141178384
Prescription arrives at: 14.519086091721029
Prescription arrives at: 14.755470493320324
Prescription arrives at: 15.431369949847943
Prescription arrives at: 15.74110894715181
Prescription arrives at: 16.67664840951203
Prescription arrives at: 16.84433068867127
Prescription arrives at: 18.22280838677